In [2]:
# Importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

In [3]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
submission_data = pd.read_csv("sample_submission_UVKGLZE.csv")

In [4]:
train_data.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [5]:
train_data.shape

(20972, 9)

In [6]:
test_data.head()

,ID,TITLE,ABSTRACT
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...


In [7]:
# checking missing values

train_data.isnull().sum()

ID                      0
TITLE                   0
ABSTRACT                0
Computer Science        0
Physics                 0
Mathematics             0
Statistics              0
Quantitative Biology    0
Quantitative Finance    0
dtype: int64

In [8]:
labels = ['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']

for col in labels:
    print(col,':\n',train_data[col].value_counts())

Computer Science :
 0    12378
1     8594
Name: Computer Science, dtype: int64
Physics :
 0    14959
1     6013
Name: Physics, dtype: int64
Mathematics :
 0    15354
1     5618
Name: Mathematics, dtype: int64
Statistics :
 0    15766
1     5206
Name: Statistics, dtype: int64
Quantitative Biology :
 0    20385
1      587
Name: Quantitative Biology, dtype: int64
Quantitative Finance :
 0    20723
1      249
Name: Quantitative Finance, dtype: int64


In [9]:
dic = {'CS' :8594,'Phy' :6013,'Math' :5618,'Stats' :5206,'QB' :587,'QF' :249}

values = dic.values()
total = sum(values)
percent_values = [value * 100. / total for value in values]
print(percent_values)

[32.71785891041992, 22.891841474092967, 21.388053451098337, 19.819545437240645, 2.234743213918605, 0.9479575132295276]


From the above 2 cells I can conclude that my dataset is highly imbalanced, especially last 2 features

In [10]:
train_data['content'] = train_data['TITLE'] + train_data['ABSTRACT']
test_data['content'] = test_data['TITLE'] + test_data['ABSTRACT']

In [11]:
train_data.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,content
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0,Reconstructing Subject-Specific Effect Maps P...
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0,Rotation Invariance Neural Network Rotation i...
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0,Spherical polyharmonics and Poisson kernels fo...
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0,A finite element approximation for the stochas...
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0,Comparative study of Discrete Wavelet Transfor...


In [12]:
train_data['content'][0]

"Reconstructing Subject-Specific Effect Maps  Predictive models allow subject-specific inference when analyzing disease\nrelated alterations in neuroimaging data. Given a subject's data, inference can\nbe made at two levels: global, i.e. identifiying condition presence for the\nsubject, and local, i.e. detecting condition effect on each individual\nmeasurement extracted from the subject's data. While global inference is widely\nused, local inference, which can be used to form subject-specific effect maps,\nis rarely used because existing models often yield noisy detections composed of\ndispersed isolated islands. In this article, we propose a reconstruction\nmethod, named RSM, to improve subject-specific detections of predictive\nmodeling approaches and in particular, binary classifiers. RSM specifically\naims to reduce noise due to sampling error associated with using a finite\nsample of examples to train classifiers. The proposed method is a wrapper-type\nalgorithm that can be used w

In [13]:
test_data.head()

,ID,TITLE,ABSTRACT,content
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...,Closed-form Marginal Likelihood in Gamma-Poiss...
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...,Laboratory mid-IR spectra of equilibrated and ...
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...,Case For Static AMSDU Aggregation in WLANs Fr...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...,The $Gaia$-ESO Survey: the inner disk intermed...
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...,Witness-Functions versus Interpretation-Functi...


In [14]:
test_data['content'][0]

'Closed-form Marginal Likelihood in Gamma-Poisson Matrix Factorization  We present novel understandings of the Gamma-Poisson (GaP) model, a\nprobabilistic matrix factorization model for count data. We show that GaP can\nbe rewritten free of the score/activation matrix. This gives us new insights\nabout the estimation of the topic/dictionary matrix by maximum marginal\nlikelihood estimation. In particular, this explains the robustness of this\nestimator to over-specified values of the factorization rank, especially its\nability to automatically prune irrelevant dictionary columns, as empirically\nobserved in previous work. The marginalization of the activation matrix leads\nin turn to a new Monte Carlo Expectation-Maximization algorithm with favorable\nproperties.\n'

In [15]:
train_data_copy = train_data.copy()
test_data_copy = test_data.copy()

In [16]:
train_data.drop(['TITLE','ABSTRACT'], axis=1, inplace=True)
test_data.drop(['ID','TITLE', 'ABSTRACT'], axis=1, inplace=True)

In [17]:
train_data.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,content
0,1,1,0,0,0,0,0,Reconstructing Subject-Specific Effect Maps P...
1,2,1,0,0,0,0,0,Rotation Invariance Neural Network Rotation i...
2,3,0,0,1,0,0,0,Spherical polyharmonics and Poisson kernels fo...
3,4,0,0,1,0,0,0,A finite element approximation for the stochas...
4,5,1,0,0,1,0,0,Comparative study of Discrete Wavelet Transfor...


In [18]:
X = train_data.loc[:,['content']]
y = train_data.loc[:,labels]

In [19]:
X

,content
0,Reconstructing Subject-Specific Effect Maps P...
1,Rotation Invariance Neural Network Rotation i...
2,Spherical polyharmonics and Poisson kernels fo...
3,A finite element approximation for the stochas...
4,Comparative study of Discrete Wavelet Transfor...
...,...
20967,Contemporary machine learning: a guide for pra...
20968,Uniform diamond coatings on WC-Co hard alloy c...
20969,Analysing Soccer Games with Clustering and Con...
20970,On the Efficient Simulation of the Left-Tail o...


In [20]:
y

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
4,1,0,0,1,0,0
...,...,...,...,...,...,...
20967,1,1,0,0,0,0
20968,0,1,0,0,0,0
20969,1,0,0,0,0,0
20970,0,0,1,1,0,0


In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=5555, shuffle=True)

In [22]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((18874, 1), (2098, 1), (18874, 6), (2098, 6))

In [23]:
y_test.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
17273,0,1,1,0,0,0
3323,0,1,0,0,0,0
14786,0,0,1,0,0,0
2030,1,0,0,0,0,0
13741,1,0,0,0,0,0


In [24]:
y_test.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

In [25]:
y1 = np.array(y_train)
y2 = np.array(y_test)

## Text Preprocessing

In [26]:
#Removing Punctuations

X_train.replace('[^a-zA-Z]',' ', regex=True, inplace=True)
X_test.replace('[^a-zA-Z]',' ', regex=True, inplace=True)

test_data.replace('[^a-zA-Z]',' ', regex=True, inplace=True)

In [27]:
#Converting to lower case characters

for index in X_train.columns:
    X_train[index] = X_train[index].str.lower()

for index in X_test.columns:
    X_test[index] = X_test[index].str.lower()

for index in test_data.columns:
    test_data[index] = test_data[index].str.lower()

In [28]:
X_train

,content
7031,determinants of public cooperation in multiple...
6054,basis adaptive sample efficient polynomial cha...
17643,statistics of turbulence in the energy contain...
11829,strong comparison principle for the fractional...
15542,phase limitations of zames falb multipliers p...
...,...
18424,analyzing the robustness of nearest neighbors ...
17900,coherent control of flexural vibrations in dua...
13516,attention set based metric learning for video ...
7646,pluripotential theory on the support of closed...


In [29]:
X_test

,content
0,grassmannian flows and applications to nonline...
1,improved point source detection in crowded fie...
2,normality and related properties of forcing al...
3,implementation of smart contracts using hybrid...
4,superminhash a new minwise hashing algorithm...
...,...
2093,decoupled access execute on arm big little en...
2094,clustering and labelling auction fraud data a...
2095,integrable point discrete equations and evol...
2096,antenna arrays for line of sight massive mimo ...


In [30]:
test_data

,content
0,closed form marginal likelihood in gamma poiss...
1,laboratory mid ir spectra of equilibrated and ...
2,case for static amsdu aggregation in wlans fr...
3,the gaia eso survey the inner disk intermed...
4,witness functions versus interpretation functi...
...,...
8984,supporting mixed datatype matrix multiplicatio...
8985,an axiomatic basis for blackwell optimality i...
8986,genevis an interactive visualization tool fo...
8987,quantifying the causal effect of speed cameras...


In [32]:
#Removing one letter words

X_train['content'] = X_train['content'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
X_test['content'] = X_test['content'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')

test_data['content'] = test_data['content'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')

In [33]:
#Removing multiple blank spaces

X_train = X_train.replace('\s+', ' ', regex=True)
X_test = X_test.replace('\s+', ' ', regex=True)

test_data = test_data.replace('\s+', ' ', regex=True)

In [34]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package punkt to C:\Users\SHIVAM
[nltk_data]     SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\SHIVAM
[nltk_data]     SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\SHIVAM
[nltk_data]     SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\SHIVAM SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [35]:
train_lines = []
for row in range(0,X_train.shape[0]):
    train_lines.append(' '.join(str(x) for x in X_train.iloc[row,:]))

test_lines = []
for row in range(0,X_test.shape[0]):
    test_lines.append(' '.join(str(x) for x in X_test.iloc[row,:]))

predtest_lines = []
for row in range(0,test_data.shape[0]):
    predtest_lines.append(' '.join(str(x) for x in test_data.iloc[row,:]))

In [36]:
train_lines[0]

'determinants of public cooperation in multiplex networks synergies between evolutionary game theory and statistical physics have significantly improved our understanding of public cooperation in structured populations multiplex networks in particular provide the theoretical framework within network science that allows us to mathematically describe the rich structure of interactions characterizing human societies while research has shown that multiplex networks may enhance the resilience of cooperation the interplay between the overlap in the structure of the layers and the control parameters of the corresponding games has not yet been investigated with this aim we consider here the public goods game on multiplex network and we unveil the role of the number of layers and the overlap of links as well as the impact of different synergy factors in different layers on the onset of cooperation we show that enhanced public cooperation emerges only when significant edge overlap is combined wi

In [37]:
test_lines[0]

'grassmannian flows and applications to nonlinear partial differential equations we show how solutions to large class of partial differential equations with nonlocal riccati type nonlinearities can be generated from the corresponding linearized equations from arbitrary initial data it is well known that evolutionary matrix riccati equations can be generated by projecting linear evolutionary flows on stiefel manifold onto coordinate chart of the underlying grassmann manifold our method relies on extending this idea to the infinite dimensional case the key is an integral equation analogous to the marchenko equation in integrable systems that represents the coodinate chart map we show explicitly how to generate such solutions to scalar partial differential equations of arbitrary order with nonlocal quadratic nonlinearities using our approach we provide numerical simulations that demonstrate the generation of solutions to fisher kolmogorov petrovskii piskunov equations with nonlocal nonlin

In [38]:
predtest_lines[0]

'closed form marginal likelihood in gamma poisson matrix factorization we present novel understandings of the gamma poisson gap model probabilistic matrix factorization model for count data we show that gap can be rewritten free of the score activation matrix this gives us new insights about the estimation of the topic dictionary matrix by maximum marginal likelihood estimation in particular this explains the robustness of this estimator to over specified values of the factorization rank especially its ability to automatically prune irrelevant dictionary columns as empirically observed in previous work the marginalization of the activation matrix leads in turn to new monte carlo expectation maximization algorithm with favorable properties '

In [39]:
from sklearn.feature_extraction.text import CountVectorizer

countvector = CountVectorizer(ngram_range=(1,2))
X_train_cv = countvector.fit_transform(train_lines)
X_test_cv = countvector.transform(test_lines)

test_cv = countvector.transform(predtest_lines)

In [40]:
#Using TfidfVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

tfidfvector = TfidfTransformer()
X_train_tf = tfidfvector.fit_transform(X_train_cv)
X_test_tf = tfidfvector.fit_transform(X_test_cv)

test_tf = tfidfvector.fit_transform(test_cv)

In [41]:
X_train

,content
7031,determinants of public cooperation in multiple...
6054,basis adaptive sample efficient polynomial cha...
17643,statistics of turbulence in the energy contain...
11829,strong comparison principle for the fractional...
15542,phase limitations of zames falb multipliers ph...
...,...
18424,analyzing the robustness of nearest neighbors ...
17900,coherent control of flexural vibrations in dua...
13516,attention set based metric learning for video ...
7646,pluripotential theory on the support of closed...


In [42]:
y_train

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
7031,1,1,0,0,0,0
6054,0,0,1,1,0,0
17643,0,1,0,0,0,0
11829,0,0,1,0,0,0
15542,1,0,1,0,0,0
...,...,...,...,...,...,...
18424,1,0,0,1,0,0
17900,0,1,0,0,0,0
13516,1,0,0,0,0,0
7646,0,0,1,0,0,0


In [43]:
X_cv = countvector.transform(train_lines)

X_tf = tfidfvector.fit_transform(X_cv)

## VANILLA LOGISTIC REGRESSION

In [44]:
!pip install scikit-multilearn

In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier # Binary Relevance
#from skmultilearn.problem_transform import ClassifierChain

lr = LogisticRegression(class_weight='balanced',C=3.0)
clf = OneVsRestClassifier(lr)

In [47]:
X_tf.shape

(18874, 916206)

In [48]:
y_train

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
7031,1,1,0,0,0,0
6054,0,0,1,1,0,0
17643,0,1,0,0,0,0
11829,0,0,1,0,0,0
15542,1,0,1,0,0,0
...,...,...,...,...,...,...
18424,1,0,0,1,0,0
17900,0,1,0,0,0,0
13516,1,0,0,0,0,0
7646,0,0,1,0,0,0


In [49]:
clf.fit(X_tf, y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=3.0,
                                                 class_weight='balanced'))

In [50]:
y_test.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,0,1,1,0,0,0
1,0,1,0,0,0,0
2,0,0,1,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0


In [51]:
preds = clf.predict(X_test_tf)
preds

array([[0, 0, 1, 0, 0, 0],
       [1, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [0, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]])

In [52]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y2, preds))

0.6506196377502383


## Linear SVC

In [54]:
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier

model = LinearSVC(class_weight='balanced')
models = MultiOutputClassifier(model)

In [75]:
models.fit(X_tf, y_train)

MultiOutputClassifier(estimator=LinearSVC(class_weight='balanced'))

In [76]:
preds = models.predict(X_test_tf)
preds

array([[0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [0, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]], dtype=int64)

In [77]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

#print(confusion_matrix(y2,preds))
print(classification_report(y2,preds))
print(accuracy_score(y2,preds))

              precision    recall  f1-score   support

           0       0.81      0.90      0.85       859
           1       0.88      0.88      0.88       610
           2       0.84      0.81      0.83       572
           3       0.73      0.83      0.78       503
           4       0.54      0.43      0.48        44
           5       0.62      0.57      0.59        23

   micro avg       0.81      0.85      0.83      2611
   macro avg       0.74      0.74      0.73      2611
weighted avg       0.81      0.85      0.83      2611
 samples avg       0.84      0.88      0.84      2611

0.6658722592945663


## SGD Classifier

In [55]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(epsilon=0.01, max_iter=2000)
multi_target_classifier = MultiOutputClassifier(clf, n_jobs=-1)

In [56]:
multi_target_classifier.fit(X_train_tf, y_train)

MultiOutputClassifier(estimator=SGDClassifier(epsilon=0.01, max_iter=2000),
                      n_jobs=-1)

In [57]:
prediction = multi_target_classifier.predict(X_test_tf)
prediction

array([[0, 0, 1, 0, 0, 0],
       [1, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [0, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [58]:
print(accuracy_score(y2, prediction))

0.669208770257388


## SGD CLASSIFIER FOR FULL DATA

In [63]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english')) 
len(stop_words)

179

In [64]:
X.replace('[^a-zA-Z]',' ', regex=True, inplace=True)

for index in X.columns:
    X[index] = X[index].str.lower()

X['content'] = X['content'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')

X = X.replace('\s+', ' ', regex=True)

X['content'] = X['content'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

X_lines = []
for row in range(0,X.shape[0]):
    X_lines.append(' '.join(str(x) for x in X.iloc[row,:]))

In [65]:
X_cv = countvector.transform(X_lines)

X_tf = tfidfvector.fit_transform(X_cv)

In [66]:
from sklearn.linear_model import SGDClassifier
from sklearn.multioutput import MultiOutputClassifier

clf = SGDClassifier(epsilon=0.01, max_iter=2000,class_weight='balanced')
multi_target_classifier = MultiOutputClassifier(clf, n_jobs=-1)

In [67]:
multi_target_classifier.fit(X_tf, y)

MultiOutputClassifier(estimator=SGDClassifier(class_weight='balanced',
                                              epsilon=0.01, max_iter=2000),
                      n_jobs=-1)

In [69]:
y_pred = multi_target_classifier.predict(X_test_tf)
y_pred

array([[0, 1, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [0, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]], dtype=int64)

In [70]:
print(accuracy_score(y2, y_pred))

0.7612011439466159


In [71]:
y_pred_sgd = multi_target_classifier.predict(test_tf)
y_pred_sgd

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 0, 0]], dtype=int64)

In [75]:
submit = pd.DataFrame({'ID': submission_data.ID, 'Computer Science': y_pred_sgd[:,0],'Physics':y_pred_sgd[:,1],'Mathematics':y_pred_sgd[:,2],'Statistics':y_pred_sgd[:,3],'Quantitative Biology':y_pred_sgd[:,4],'Quantitative Finance':y_pred_sgd[:,5]})

In [76]:
submit.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,1,0,0,0


In [77]:
submit.to_csv('submission_sgd04.csv', index=False)